In [1]:
!curl -O http://static.decontextualize.com/gutenberg-poetry-v001.ndjson.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 52.2M    0 54466    0     0  51074      0  0:17:53  0:00:01  0:17:52 51189
  2 52.2M    2 1228k    0     0   590k      0  0:01:30  0:00:02  0:01:28  591k
 12 52.2M   12 6752k    0     0  2295k      0  0:00:23  0:00:02  0:00:21 2296k
 26 52.2M   26 14.0M    0     0  3637k      0  0:00:14  0:00:03  0:00:11 3638k
 44 52.2M   44 23.1M    0     0  4768k      0  0:00:11  0:00:04  0:00:07 4770k
 63 52.2M   63 33.3M    0     0  5749k      0  0:00:09  0:00:05  0:00:04 6994k
 85 52.2M   85 44.6M    0     0  6582k      0  0:00:08  0:00:06  0:00:02 9144k
100 52.2M  100 52.2M    0     0  7025k      0  0:00:07  0:00:07 --:--:-- 9999k


In [2]:
import gzip, json
all_lines = []
for line in gzip.open("gutenberg-poetry-v001.ndjson.gz"):
    all_lines.append(json.loads(line.strip()))

In [3]:
import random
corpus = "\n".join([line['s'] for line in random.sample(all_lines, 1000)])

In [4]:
import markovify
model = markovify.NewlineText(corpus)

ModuleNotFoundError: No module named 'markovify'

In [ ]:
for i in range(5):
    print()
    for i in range(random.randrange(1, 4)):
        print(model.make_short_sentence(30))

In [16]:
from keras.layers import LSTM, Dense, Dropout, Flatten
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.utils import np_utils

In [5]:
# Lowercase all text
text = corpus.lower()

chars = list(set(text))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

vocab_size = len(chars)
print('Vocabulary size: {}'.format(vocab_size))

Vocabulary size: 77


In [6]:
# Data preparation
X = [] # training array
Y = [] # target array

length = len(text)
seq_length = 100 # number of characters to consider before predicting a character

# Iterate over length of text and create sequences stored in X, true values stored in Y
# true values being which character would actually come after sequence stored in X
for i in range(0, length - seq_length, 1):
    sequence = text[i:i + seq_length]
    label = text[i + seq_length]
    X.append([char_indices[char] for char in sequence])
    Y.append(char_indices[label])

print('Number of sequences: {}'.format(len(X)))

Number of sequences: 39465


In [13]:
import numpy as np
from keras.utils import np_utils

C:\Users\keane\AppData\Roaming\Python\Python39\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [14]:
# Reshape dimensions
X_new = np.reshape(X, (len(X), seq_length, 1))
# Scale values
X_new = X_new/float(len(chars))
# One-hot encode Y to remove ordinal relationships
Y_new = np_utils.to_categorical(Y)

X_new.shape, Y_new.shape

((39465, 100, 1), (39465, 77))

In [17]:
model = Sequential()
# Add LSTM layer to compute output using 150 LSTM units
model.add(LSTM(150, input_shape = (X_new.shape[1], X_new.shape[2]), return_sequences = True))

# Add regularization layer to prevent overfitting.
# Dropout ignores randomly selected neurons during training ("dropped out").
# Ultimately, network becomes less sensitive to specific weights of neurons --> network is better at generalization.
model.add(Dropout(0.1))

model.add(Flatten())
# Dense layer with softmax activation function to approximate probability distribution of next best word
model.add(Dense(Y_new.shape[1], activation = 'softmax'))

# Compile model to configure learning process
# Categorical crossentropy: an example can only belong to one class
# Adam optimization algorithm updates a learning rate for each network weight iteratively as learning unfolds
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

# Use 1 epoch for sake of computational time
model.fit(X_new, Y_new, epochs = 1, verbose = 1)


1234/1234 [==============================] - 323s 255ms/step - loss: 3.0589


In [18]:
# Random start
start = np.random.randint(0, len(X)-1)
string_mapped = list(X[start])
full_string = [indices_char[value] for value in string_mapped]

# Generate text
for i in range(400):
    x = np.reshape(string_mapped, (1, len(string_mapped), 1))
    x = x / float(len(chars))
    
    pred_index = np.argmax(model.predict(x, verbose = 0))
    seq = [indices_char[value] for value in string_mapped]
    full_string.append(indices_char[pred_index])
    
    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]
    
# Combine text
newtext = ''
for char in full_string:
    newtext = newtext + char

print(newtext)

 "offense be rank," should mine be rancor?
in measured song, as of the fruitful year
with bracelets                                                                                                                                                                                                                                                                                                                                                                                                                 


In [21]:
import keras.utils as ku
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding

# Lowercase all text
text = corpus.lower()
text = text.split('\n')

# Create Tokenizer object to convert words to sequences of integers
tokenizer = Tokenizer(num_words = None, filters = '#$%&()*+-<=>@[\\]^_`{|}~\t\n', lower = False)

# Train tokenizer to the texts
tokenizer.fit_on_texts(text)
total_words = len(tokenizer.word_index) + 1

# Convert list of strings into flat dataset of sequences of tokens
sequences = []
for line in text:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        sequences.append(n_gram_sequence)

# Pad sequences to ensure equal lengths
max_seq_len = max([len(x) for x in sequences])
sequences = np.array(pad_sequences(sequences, maxlen = max_seq_len, padding = 'pre'))

# Create n-grams sequence predictors and labels
predictors, label = sequences[:, :-1], sequences[:, -1]
label = ku.to_categorical(label, num_classes = total_words)

In [24]:
# Input layer takes sequence of words as input
input_len = max_seq_len - 1
model = Sequential()
model.add(Embedding(total_words, 10, input_length = input_len))
model.add(LSTM(150))
model.add(Dropout(0.1))
model.add(Dense(total_words, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

# Use 100 epoch for efficacy
model.fit(predictors, label, epochs = 100, verbose = 1)

Epoch 1/100
197/197 [==============================] - 15s 44ms/step - loss: 7.4935
Epoch 2/100
197/197 [==============================] - 8s 39ms/step - loss: 6.9542
Epoch 3/100
197/197 [==============================] - 7s 36ms/step - loss: 6.8104
Epoch 4/100
197/197 [==============================] - 7s 37ms/step - loss: 6.7080
Epoch 5/100
197/197 [==============================] - 7s 36ms/step - loss: 6.6095
Epoch 6/100
197/197 [==============================] - 7s 36ms/step - loss: 6.4978
Epoch 7/100
197/197 [==============================] - 7s 36ms/step - loss: 6.3643
Epoch 8/100
197/197 [==============================] - 7s 36ms/step - loss: 6.2194
Epoch 9/100
197/197 [==============================] - 7s 36ms/step - loss: 6.0678
Epoch 10/100
197/197 [==============================] - 7s 36ms/step - loss: 5.9175
Epoch 11/100
197/197 [==============================] - 7s 34ms/step - loss: 5.7686
Epoch 12/100
197/197 [==============================] - 7s 34ms/step - loss: 5.6192


In [26]:
model.save("poem_model")

INFO:tensorflow:Assets written to: poem_model\assets


INFO:tensorflow:Assets written to: poem_model\assets


In [29]:
import keras

In [47]:
model2 = keras.models.load_model("poem_model")

In [39]:
# Function to generate line
def generate_line(text, next_words, max_seq_len, model):
    for j in range(next_words):
        token_list = tokenizer.texts_to_sequences([text])[0]
        token_list = pad_sequences([token_list], maxlen = max_seq_len - 1, padding = 'pre')
        predicted = model.predict(token_list, verbose = 0)
        classes = np.argmax(predicted,axis=1)
        
        output_word = ''
        for word, index in tokenizer.word_index.items():
            if index == classes:
                output_word = word
                break
        text += ' ' + output_word
    return text

In [48]:
generate_line("peace always", 20, max_seq_len, model2)

'peace always the grass that wells from a feathered throat and the waist! of my ground. sterilis to my cheeks, and i'